In [214]:
import openai as client

# assistant = client.beta.assistants.create(
#    name="Book Assistant2",
#    instructions=(
#        "You help users with their question on the files they upload."
#    ),
#    model="gpt-4o-mini",
#    tools=[{"type": "file_search"}],
# )

assistant_id = "asst_PMX0zlhX3Hfg2P35jsAvJK3g"

In [235]:
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "I want you to help me with this file",
        }
    ]
)
thread

Thread(id='thread_7yIHcJnhoSbDL8o39y6f7xbB', created_at=1735143550, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))

In [237]:
file = client.files.create(
    file=client.file_from_path("./files/chapter_one.txt"), purpose="assistants"
)
file

FileObject(id='file-NujD8851jtVoEfTiEuZjmL', bytes=35537, created_at=1735143561, filename='chapter_one.txt', object='file', purpose='assistants', status='processed', status_details=None)

In [236]:
client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Please refer to the uploaded file for my question.",
    attachments=[
        {
            "file_id": file.id,
            "tools": [
                {
                    "type": "file_search",
                }
            ],
        }
    ],
)

Message(id='msg_qNuo3sKAh3RO7C0QLgunLf5D', assistant_id=None, attachments=[Attachment(file_id='file-9CvNWCTvZXQAcovovszJr9', tools=[AttachmentToolAssistantToolsFileSearchTypeOnly(type='file_search')])], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Please refer to the uploaded file for my question.'), type='text')], created_at=1735143557, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_7yIHcJnhoSbDL8o39y6f7xbB')

In [242]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id,
)
run

Run(id='run_VUsPCVW9uDUVGxqYV4VQHNf3', assistant_id='asst_PMX0zlhX3Hfg2P35jsAvJK3g', cancelled_at=None, completed_at=None, created_at=1735143592, expires_at=1735144192, failed_at=None, incomplete_details=None, instructions='You help users with their question on the files they upload.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o-mini', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_7yIHcJnhoSbDL8o39y6f7xbB', tool_choice='auto', tools=[FileSearchTool(type='file_search', file_search=FileSearch(max_num_results=None, ranking_options=FileSearchRankingOptions(score_threshold=0.0, ranker='default_2024_08_21')))], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={})

In [217]:
def get_run(run_id, thread_id):
    return client.beta.threads.runs.retrieve(
        run_id=run_id,
        thread_id=thread_id,
    )


def send_message(thread_id, content):
    return client.beta.threads.messages.create(
        thread_id=thread_id, role="user", content=content
    )


def get_messages(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    messages = list(messages)
    messages.reverse()
    for message in messages:
        print(f"{message.role}: {message.content[0].text.value}")
        for annotation in message.content[0].text.annotations:
            print(f"Source: {annotation.file_citation}")

In [243]:
get_run(run.id, thread.id).status

'completed'

In [244]:
get_messages(thread.id)

user: I want you to help me with this file
user: Please refer to the uploaded file for my question.
assistant: Please specify what question you have regarding the uploaded file, and I'll assist you accordingly.
user: I want to know where does winston lives and how his apartment looks like.
assistant: Winston lives in a place called Victory Mansions, which is described in a rather disparaging manner. The building is characterized by a glass door through which Winston enters, likely letting in gritty dust. The hallway has an unpleasant smell of boiled cabbage, and there is a large poster depicting the face of Big Brother at one end. The elevators are usually out of order, which requires Winston to walk seven flights of stairs to reach his apartment. Winston's physical condition (he has a varicose ulcer) makes this climb difficult, and he needs to rest several times on the way up【7:0†source】.

Inside Winston's flat, he experiences a mix of oppressive technology and stark living conditions

In [241]:
send_message(
    thread_id=thread.id,
    content=(
        "I want to know where does winston lives and how his apartment looks"
        " like."
    ),
)

Message(id='msg_9hW7ZwUYlqVCgZunINDVRA2D', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='I want to know where does winston lives and how his apartment looks like.'), type='text')], created_at=1735143582, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_7yIHcJnhoSbDL8o39y6f7xbB')

In [ ]:
from langchain.document_loaders import WebBaseLoader
from langchain.tools import DuckDuckGoSearchResults, WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
import json
import time


def duckduckgo_search(inputs):
    query = inputs["query"]
    search = DuckDuckGoSearchResults()
    return search.run(query)


def wikipedia_search(inputs):
    query = inputs["query"]
    wiki_api_wrapper = WikipediaAPIWrapper()
    wiki = WikipediaQueryRun(api_wrapper=wiki_api_wrapper)
    return wiki.run(query)


def web_scraping(inputs):
    url = inputs["url"]
    loader = WebBaseLoader([url])
    docs = loader.load()
    return "\n\n".join([doc.page_content for doc in docs])


def save_to_txt(inputs):
    text = inputs["text"]
    with open("research_results.txt", "w") as file:
        file.write(text)
    return "Research results saved to research_results.txt"


functions_map = {
    "DuckDuckGoSearchTool": duckduckgo_search,
    "WikipediaSearchTool": wikipedia_search,
    "WebScrapingTool": web_scraping,
    "SaveToTXTTOOL": save_to_txt,
}


functions = [
    {
        "type": "function",
        "function": {
            "name": "DuckDuckGoSearchTool",
            "description": (
                "Perform web searches using DuckDuckGo. Input a query string."
            ),
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The query you will search for",
                    },
                },
                "required": ["query"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "WikipediaSearchTool",
            "description": (
                "Perform searches on Wikipedia. Input a query string."
            ),
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": (
                            "The query you will search for on Wikipedia"
                        ),
                    },
                },
                "required": ["query"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "WebScrapingTool",
            "description": "Scrape content from a given URL.",
            "parameters": {
                "type": "object",
                "properties": {
                    "url": {
                        "type": "string",
                        "description": (
                            "The URL of the website you want to scrape"
                        ),
                    },
                },
                "required": ["url"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "SaveToTXTTOOL",
            "description": "Use this tool to save the content as a .txt file.",
            "parameters": {
                "type": "object",
                "properties": {
                    "text": {
                        "type": "string",
                        "description": "The text you will save to a file.",
                    },
                },
                "required": ["text"],
            },
        },
    },
]

In [3]:
from openai import OpenAI

client = OpenAI()

assistant = client.beta.assistants.create(
    name="Search Assistant",
    instructions=(
        "You are a research expert. Use Wikipedia or DuckDuckGo to"
        " gather information. Scrape content from relevant links and"
        " provide comprehensive answers. Save findings to a .txt file."
    ),
    tools=functions,
    model="gpt-4o-mini",
)

assistant

Assistant(id='asst_IUuPboPffsaZWTfPupKiOtNt', created_at=1735388080, description=None, instructions='You are a research expert. Use Wikipedia or DuckDuckGo to gather information. Scrape content from relevant links and provide comprehensive answers. Save findings to a .txt file.', metadata={}, model='gpt-4o-mini', name='Search Assistant', object='assistant', tools=[FunctionTool(function=FunctionDefinition(name='DuckDuckGoSearchTool', description='Perform web searches using DuckDuckGo. Input a query string.', parameters={'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'The query you will search for'}}, 'required': ['query']}, strict=False), type='function'), FunctionTool(function=FunctionDefinition(name='WikipediaSearchTool', description='Perform searches on Wikipedia. Input a query string.', parameters={'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'The query you will search for on Wikipedia'}}, 'required': ['query']}, strict=Fal